In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/RecFldGrn


# RecName Settings

In [2]:
import os

rec_folder = 'data/ProcData/RecFolder'

RecName_List = os.listdir(rec_folder)
RecName_List

['PNSect', 'Diag', 'Smoking', 'EC', 'PN', 'A1C', 'PNSectSent', 'P']

In [3]:
import os

RecName = 'P'

# the folder to save the Record.
rec_folder = 'data/ProcData/RecFolder'
fullrec_folder = os.path.join(rec_folder, RecName)
print(fullrec_folder, '<----- fullrec_folder to save the records')

data/ProcData/RecFolder/P <----- fullrec_folder to save the records


## Load One

In [4]:
from recfldgrn.datapoint import load_df_data_from_folder, DataPoint

####################
RID = 'PID' # Record ID Name, always be PhyID
IDValue = 'P5'   # PhyID value
######################
data_folder = rec_folder
RANGE_SIZE = 10000

# print(data_folder)
Pat = DataPoint(RID, IDValue, data_folder, RANGE_SIZE)
Pat

<PID [P5]>

In [5]:
print(Pat)
dfx = Pat.get_df_rec('P')
dfx

<PID [P5]>


,PID,age,basicInfo
0,P5,73,Female&D


## Load Whole

In [6]:
# import this function
from recfldgrn.datapoint import load_df_data_from_folder
# give the fullrec_folder
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder)
df

data/ProcData/RecFolder/P


,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


# FlGrn: PatEcInfo-InfoGrn

1. RecLevel: 'P' (One and Only One)
2. SynFld: 'BasicInfo'
3. GrnName: 'InfoGrn'
4. Rec2ColList_Dict: {'P': ['age', 'basicInfo']}


## Settings

In [33]:
# ['Age18', 'GenderMale', 'RaceB']
# grain_list

In [32]:
# P@Age
# P@BirthDate


In [34]:
# mean EC hours
# P@EChoursInfo --> ['mean', 'var']

In [35]:
###################
RecLevel = 'P' # grain_list as whole is in P RecLevel. 
RecLevelID = f'{RecLevel}ID' # (One and Only One)

SynFld = 'PatEcInfo'
GrnName = 'InfoGrn'


# P@PatEcInfo --> grain_list 
# input: RecFld
Rec2FldList_Dict = {
    'P': ['age', 'basicInfo'],
    'EC': ['DT_min', 'DT_max'], ## IMPORTANT: why we use EC here as it is not in P level. 
} # this should be order.

prefix_ids = [] # P-level. # Empty, but this is important, which will be shown in next notebook.
focal_ids = [RecLevelID]

rec_folder = 'data/ProcData/RecFolder'
###################

recfldgrn = f'{RecLevel}@{SynFld}-{GrnName}'
recfldgrn

'P@PatEcInfo-InfoGrn'

## Get df_whole

In [36]:
import pandas as pd
from functools import reduce

######## we will remove this to graintools.py file
def get_df_whole_from_settings(RecLevel, RecLevelID, 
                               SynFld, GrnName, Rec2FldList_Dict, 
                               prefix_ids, focal_ids, rec_folder,
                               **kwargs
                              ):
    ## get the base IDs in case you need it. # eg: RecLevel: BMI, RecName: P.
    # Way 1
    # df_base = Pat.get_df_rec(RecLevel)[prefix_ids + focal_ids]
    # Way 2
    df_base = load_df_data_from_folder(os.path.join(rec_folder, RecLevel))[prefix_ids + focal_ids]


    L = []
    for RecName, FldList in Rec2FldList_Dict.items():
        #print(RecLevelID, RecName, FldList)
        full_cols = prefix_ids + focal_ids + FldList
        # Way 1
        # df_whole = Pat.get_df_rec(RecName)[full_cols]
        # Way 2
        df_whole = load_df_data_from_folder(os.path.join(rec_folder, RecName))[full_cols]


        # make sure RecLevelID in df_whole.columns
        if RecLevelID not in df_whole.columns:
            print('Use this...')
            on_cols = [i for i in df_base.columns if i in df_whole.columns]
            df_whole = pd.merge(df_base, df_whole, on = on_cols, how = 'left')


        # df_whole_dict[RecName] = df_whole

        df_input_groups = pd.DataFrame([{RecLevelID: RecLevelIDValue, RecName: df_input} 
                                         for RecLevelIDValue, df_input in df_whole.groupby(RecLevelID)])

        L.append(df_input_groups)


    # Merge the dataframes in the list using reduce
    df_whole = reduce(lambda left, right: pd.merge(left, right, on=RecLevelID), L)
    
    # df_base
    df_whole = pd.merge(df_base, df_whole, on = focal_ids, how = 'left') 
    
    return df_whole


In [37]:
from recfldgrn.graintools import get_df_whole_from_settings

df_whole = get_df_whole_from_settings(RecLevel, RecLevelID, 
                                       SynFld, GrnName, Rec2FldList_Dict, 
                                       prefix_ids, focal_ids, rec_folder
                                      )
df_whole

,PID,P,EC
0,P0,PID age basicInfo 0 P0 52 Male&B,PID DT_min DT_max 0 P0 2023-03-18...
1,P1,PID age basicInfo 1 P1 21 Male&B,PID DT_min DT_max 34 P1 2023-03-18...
2,P2,PID age basicInfo 2 P2 23 Female&B,PID DT_min DT_max 59 P2 2023-03-18...
3,P3,PID age basicInfo 3 P3 77 Female&B,PID DT_min DT_max 85 P3 2023-03-...
4,P4,PID age basicInfo 4 P4 23 Male&A,PID DT_min DT_max 172 P4 2023-03-...
5,P5,PID age basicInfo 5 P5 73 Female&D,PID DT_min DT_max 194 P5 2023-03-...
6,P6,PID age basicInfo 6 P6 34 Female&A,PID DT_min DT_max 217 P6 2023-03-...
7,P7,PID age basicInfo 7 P7 76 Male&C,PID DT_min DT_max 224 P7 2023-03-...


In [38]:
Rec2FldList_Dict

{'P': ['age', 'basicInfo'], 'EC': ['DT_min', 'DT_max']}

In [40]:
df_dp = df_whole.iloc[0]
df_dp['P']

,PID,age,basicInfo
0,P0,52,Male&B


In [43]:
dfx = df_dp['EC']
dfx.head()

,PID,DT_min,DT_max
0,P0,2023-03-18,2023-04-16
1,P0,2023-03-19,2023-04-05
2,P0,2023-03-19,2023-04-16
3,P0,2023-03-20,2023-04-16
4,P0,2023-03-20,2023-04-15


## Get df_input

In [44]:
# one sample
df_input = df_whole.iloc[6]
# df_input.to_dict()
df_input[prefix_ids + focal_ids]

PID    P6
Name: 6, dtype: object

In [45]:
df_input['P']

,PID,age,basicInfo
6,P6,34,Female&A


In [46]:
df_input['EC']

,PID,DT_min,DT_max
217,P6,2023-03-19,2023-04-12
218,P6,2023-03-19,2023-03-28
219,P6,2023-03-19,2023-04-04
220,P6,2023-03-21,2023-04-09
221,P6,2023-03-23,2023-04-04
222,P6,2023-03-25,2023-03-25
223,P6,2023-04-05,2023-04-13


In [47]:
# assert the df_input
assert RecLevelID in df_input
for RecName, FldList in Rec2FldList_Dict.items():
    assert RecName in df_input
    df_recinput = df_input[RecName]
    assert set(FldList).issubset(set(df_recinput.columns))

## Get grain_tkn_fn

In [49]:
# from recfldgrn.graintools import func_convert_Num_factory

import numpy as np

def func_convert_Num_factory(start, end, Min, Max, scale):
    
    def func_convert_NUMgrn(v):

        miss, more, less, bottom, top = 0, 0, 0, 0, 0; base = 1
        
        if pd.isna(v) == True:
            miss = 1; base = 0
            num_embed = int((end - start) / scale)
            x = np.zeros(num_embed + 1)# .astype(int)
            x = list(x)
            wgt = [miss, more, less, bottom, top, base] + x
            tkn = ['miss', 'more', 'less', 'bottom', 'top', 'base'] + [f'B{start + i*scale}+' for i in range(len(x))]
            return tkn, wgt
        
        else:
            if type(v) == str:
                if v[0] not in '<>-': v = float(v) # No special symbols
                elif v[0] == '<': less = 1; v = float(v[1:]) # if < is here
                elif v[0] == '>': more = 1; v = float(v[1:]) # if > is here
                
            if v < start:  bottom = (start - v) / (start - Min); base = 0; v = start
            elif v > end: top = (v-end) / (Max-end); v = end

            num_embed = int((end - start) / scale)
            x = np.zeros(num_embed + 1)# .astype(f)
            num = int((v - start) / scale)
            # print(num)
            x[:num] = 1
            # print(v, start, scale,  v - start, num)
            # print( (v-start) - num * scale)
            x[num] = ((v-start) - num * scale) / scale
            x = x.round(4)
            # print((v - start) % scale, v, start, scale)
            # print(x[num])
            # print(x)
            x = list(x) # value part

            wgt = [miss, more, less, bottom, top, base] + x
            tkn = ['miss', 'more', 'less', 'bottom', 'top', 'base'] +[f'B{start + i*scale}+' for i in range(len(x))]
            return tkn, wgt
        
    return func_convert_NUMgrn


In [50]:
import itertools
import inspect

def grain_tkn_fn(df_input):
    
    buffer_dict = {}
    
    # Patient Part: 
    df_recinput = df_input['P'][['age', 'basicInfo']] # this special because xxx.
    
    # P@Age
    fld = 'age'
    start, end, Min, Max, scale = [18, 80, 1, 120, 5]
    func_convert_Num_wgt = func_convert_Num_factory(start, end, Min, Max, scale)
    value = df_recinput[fld].iloc[0]
    results = func_convert_Num_wgt(value)
    li_value_tkn = [f'{fld}_{i}' for i in results[0]]
    li_value_wgt = results[1]
    # print(len(li_value_tkn), len(li_value_wgt))
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'tkn': li_value_tkn, 'wgt': li_value_wgt, 'fld':li_value_fld}
    
    # P@basicInfo
    fld = 'basicInfo'
    basicInfo_series = df_recinput[fld]# .mean()
    value = basicInfo_series.iloc[0]
    li_value_tkn = value.split('&')
    li_value_wgt = [1] * len(li_value_tkn)
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'tkn': li_value_tkn, 'wgt': li_value_wgt, 'fld':li_value_fld}
    
    # EC Part
    df_recinput = df_input['EC'][['DT_min', 'DT_max']]
    
    # P@ECNum
    fld = 'ecNum'
    value = len(df_recinput)
    li_value_tkn = [f'{fld}_{value}' ]
    li_value_wgt = [1] * len(li_value_tkn)
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'tkn': li_value_tkn, 'wgt': li_value_wgt, 'fld':li_value_fld}
    
    # P@ecDays
    fld = 'ecDays'
    start, end, Min, Max, scale = [0, 100, 0, 500, 7]
    func_convert_Num_wgt = func_convert_Num_factory(start, end, Min, Max, scale)
    value = (df_recinput['DT_max'] - df_recinput['DT_min']).mean().total_seconds() / 60 / 60 / 24
    results = func_convert_Num_wgt(value)
    li_value_tkn = [f'{fld}{i}' for i in results[0]]
    li_value_wgt = results[1]
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'tkn': li_value_tkn, 'wgt': li_value_wgt, 'fld':li_value_fld}
    
    # summary
    output = {}
    for i in ['tkn', 'wgt', 'fld']:
        # output[i] = sum(, [])
        output[i.split('_')[-1]] = list(itertools.chain(*[d[i] for fld, d in buffer_dict.items()]))
        
    return output

grain_tkn_fn_string = inspect.getsource(grain_tkn_fn)
# print(grain_str_fn_string)

In [51]:
output = grain_tkn_fn(df_input)
for name, grn_list in output.items():
    print('\n')
    print(name)
    print(grn_list)
    print(len(grn_list))



tkn
['age_miss', 'age_more', 'age_less', 'age_bottom', 'age_top', 'age_base', 'age_B18+', 'age_B23+', 'age_B28+', 'age_B33+', 'age_B38+', 'age_B43+', 'age_B48+', 'age_B53+', 'age_B58+', 'age_B63+', 'age_B68+', 'age_B73+', 'age_B78+', 'Female', 'A', 'ecNum_7', 'ecDaysmiss', 'ecDaysmore', 'ecDaysless', 'ecDaysbottom', 'ecDaystop', 'ecDaysbase', 'ecDaysB0+', 'ecDaysB7+', 'ecDaysB14+', 'ecDaysB21+', 'ecDaysB28+', 'ecDaysB35+', 'ecDaysB42+', 'ecDaysB49+', 'ecDaysB56+', 'ecDaysB63+', 'ecDaysB70+', 'ecDaysB77+', 'ecDaysB84+', 'ecDaysB91+', 'ecDaysB98+']
43


wgt
[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1.0, 0.7959, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
43


fld
['age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'basicInfo', 'basicInfo', 'ecNum', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays',

## Get Grain_List

In [52]:
recfldgrn = f'{RecLevel}@{SynFld}-{GrnName}'
recfldgrn

'P@PatEcInfo-InfoGrn'

In [18]:
df_whole[recfldgrn] = df_whole.apply(lambda df_input: grain_tkn_fn(df_input), axis = 1)
for i in ['tkn', 'wgt', 'fld']:
    df_whole[f'{recfldgrn}_{i}'] = df_whole[recfldgrn].apply(lambda x: x[i])
df_whole

,PID,P,EC,P@PatEcInfo-InfoGrn,P@PatEcInfo-InfoGrn_tkn,P@PatEcInfo-InfoGrn_wgt,P@PatEcInfo-InfoGrn_fld
0,P0,PID age basicInfo 0 P0 52 Male&B,PID DT_min DT_max 0 P0 2023-03-18...,"{'tkn': ['age_miss', 'age_more', 'age_less', '...","[age_miss, age_more, age_less, age_bottom, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."
1,P1,PID age basicInfo 1 P1 21 Male&B,PID DT_min DT_max 34 P1 2023-03-18...,"{'tkn': ['age_miss', 'age_more', 'age_less', '...","[age_miss, age_more, age_less, age_bottom, age...","[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
2,P2,PID age basicInfo 2 P2 23 Female&B,PID DT_min DT_max 59 P2 2023-03-18...,"{'tkn': ['age_miss', 'age_more', 'age_less', '...","[age_miss, age_more, age_less, age_bottom, age...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
3,P3,PID age basicInfo 3 P3 77 Female&B,PID DT_min DT_max 85 P3 2023-03-...,"{'tkn': ['age_miss', 'age_more', 'age_less', '...","[age_miss, age_more, age_less, age_bottom, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."
4,P4,PID age basicInfo 4 P4 23 Male&A,PID DT_min DT_max 172 P4 2023-03-...,"{'tkn': ['age_miss', 'age_more', 'age_less', '...","[age_miss, age_more, age_less, age_bottom, age...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
5,P5,PID age basicInfo 5 P5 73 Female&D,PID DT_min DT_max 194 P5 2023-03-...,"{'tkn': ['age_miss', 'age_more', 'age_less', '...","[age_miss, age_more, age_less, age_bottom, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."
6,P6,PID age basicInfo 6 P6 34 Female&A,PID DT_min DT_max 217 P6 2023-03-...,"{'tkn': ['age_miss', 'age_more', 'age_less', '...","[age_miss, age_more, age_less, age_bottom, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0....","[age, age, age, age, age, age, age, age, age, ..."
7,P7,PID age basicInfo 7 P7 76 Male&C,PID DT_min DT_max 224 P7 2023-03-...,"{'tkn': ['age_miss', 'age_more', 'age_less', '...","[age_miss, age_more, age_less, age_bottom, age...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[age, age, age, age, age, age, age, age, age, ..."


## Get Embed Info

In [19]:
def generate_grain_vocab_info(s):
    '''
        s: a Series (or a column of a dataframe) of all rec@fld value's grain_list.
            eg. grain_list: a list of tokens from raw sentence. 
        
        output: a dict of idx2v, v2dix, v2freq.
    '''
    value_list = itertools.chain(*s.to_list())
    d = pd.Series(value_list).value_counts().to_dict()
    v2freq = {'_padding': 0}
    if '_missing' in d:
        v2freq['_missing'] = d.pop('_missing') # I should have some prefix here. 
    for k, v in d.items(): v2freq[k] = v

    idx2v = {idx: v for idx, v in enumerate(v2freq.keys())}
    v2idx = {v:k for k, v in idx2v.items()}
    
    return {'idx2v': idx2v, 'v2idx': v2idx, 'v2freq': v2freq}

### Embed Info

In [20]:
EmbedDict = {}

### tkn embed

In [23]:
sfx = 'tkn'
embed_type = 'CateEmbed'

recfldgrn_sfx = f'{recfldgrn}_{sfx}'
print(recfldgrn_sfx)
s = df_whole[recfldgrn_sfx] 
Vocab = generate_grain_vocab_info(s) 
v2idx = Vocab['v2idx']
vocab_size = len(v2idx)
tknz = None
print(v2idx)

d = {'embed_type':embed_type, 
     'recfldgrn_sfx':recfldgrn_sfx,
     'vocab_size': vocab_size,
     'Vocab': Vocab,
     'tknz': tknz}

df_whole[f'{recfldgrn}_{sfx}idx'] =  df_whole[f'{recfldgrn}_{sfx}'].apply(lambda x: [ v2idx[i] for i in x])

EmbedDict[sfx] = d

P@PatEcInfo-InfoGrn_tkn
{'_padding': 0, 'age_miss': 1, 'ecDaysmiss': 2, 'ecDaysless': 3, 'ecDaysbottom': 4, 'ecDaystop': 5, 'age_more': 6, 'ecDaysB0+': 7, 'ecDaysB7+': 8, 'ecDaysB14+': 9, 'ecDaysB21+': 10, 'ecDaysB28+': 11, 'ecDaysB35+': 12, 'ecDaysB42+': 13, 'ecDaysB49+': 14, 'ecDaysB56+': 15, 'ecDaysB63+': 16, 'ecDaysB70+': 17, 'ecDaysB77+': 18, 'ecDaysB84+': 19, 'ecDaysB91+': 20, 'ecDaysB98+': 21, 'ecDaysmore': 22, 'ecDaysbase': 23, 'age_B18+': 24, 'age_B43+': 25, 'age_less': 26, 'age_bottom': 27, 'age_top': 28, 'age_base': 29, 'age_B23+': 30, 'age_B28+': 31, 'age_B38+': 32, 'age_B33+': 33, 'age_B48+': 34, 'age_B53+': 35, 'age_B58+': 36, 'age_B63+': 37, 'age_B68+': 38, 'age_B73+': 39, 'age_B78+': 40, 'B': 41, 'Male': 42, 'Female': 43, 'A': 44, 'D': 45, 'C': 46, 'ecNum_7': 47, 'ecNum_23': 48, 'ecNum_34': 49, 'ecNum_22': 50, 'ecNum_87': 51, 'ecNum_26': 52, 'ecNum_25': 53, 'ecNum_1': 54}


### fldtype embed

In [24]:
sfx = 'fld'
embed_type = 'CateEmbed'

recfldgrn_sfx = f'{recfldgrn}_{sfx}'
print(recfldgrn_sfx)
s = df_whole[recfldgrn_sfx] 
Vocab = generate_grain_vocab_info(s) 
v2idx = Vocab['v2idx']
vocab_size = len(v2idx)
print(v2idx)

d = {'embed_type':embed_type, 
     'recfldgrn_sfx':recfldgrn_sfx,
     'vocab_size': vocab_size,
     'Vocab': Vocab,
     'tknz': tknz}

df_whole[f'{recfldgrn}_{sfx}idx'] =  df_whole[f'{recfldgrn}_{sfx}'].apply(lambda x: [v2idx[i] for i in x])

EmbedDict[sfx] = d

P@PatEcInfo-InfoGrn_fld
{'_padding': 0, 'ecDays': 1, 'age': 2, 'basicInfo': 3, 'ecNum': 4}


In [25]:
# EmbedInfo_Dict[sfx] 
d

{'embed_type': 'CateEmbed',
 'recfldgrn_sfx': 'P@PatEcInfo-InfoGrn_fld',
 'vocab_size': 5,
 'Vocab': {'idx2v': {0: '_padding',
   1: 'ecDays',
   2: 'age',
   3: 'basicInfo',
   4: 'ecNum'},
  'v2idx': {'_padding': 0, 'ecDays': 1, 'age': 2, 'basicInfo': 3, 'ecNum': 4},
  'v2freq': {'_padding': 0,
   'ecDays': 168,
   'age': 152,
   'basicInfo': 16,
   'ecNum': 8}},
 'tknz': None}

## Group to P Level

In [26]:
final_cols = [RecLevelID] + [i for i in df_whole.columns if 'idx' == i[-3:] or 'wgt' == i[-3:]]
df_tensor = df_whole[final_cols]
df_tensor

,PID,P@PatEcInfo-InfoGrn_wgt,P@PatEcInfo-InfoGrn_tknidx,P@PatEcInfo-InfoGrn_fldidx
0,P0,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,P1,"[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,P2,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,P3,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,P4,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
5,P5,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
6,P6,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
7,P7,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [27]:
df_tensor_fnl = df_tensor
df_tensor_fnl.head()

,PID,P@PatEcInfo-InfoGrn_wgt,P@PatEcInfo-InfoGrn_tknidx,P@PatEcInfo-InfoGrn_fldidx
0,P0,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,P1,"[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,P2,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,P3,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,P4,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [28]:
prefix_recs = [i.replace('ID', '') for i in prefix_ids]
prefix_recfldgrn = '-'.join(prefix_recs) + '-' + recfldgrn if len(prefix_recs) > 0 else recfldgrn
prefix_recfldgrn

'P@PatEcInfo-InfoGrn'

## Save

In [29]:
Info = {}

Info['RecLevel'] = RecLevel
Info['RecLevelID'] = RecLevelID
Info['SynFld'] = SynFld
Info['GrnName'] = GrnName
Info['Rec2FldList_Dict'] = Rec2FldList_Dict
Info['prefix_ids'] = prefix_ids
Info['focal_ids'] = focal_ids
Info['rec_folder'] = rec_folder
Info['recfldgrn'] = recfldgrn
Info['prefix_recfldgrn'] = prefix_recfldgrn
Info['grain_tkn_fn_string'] = grain_tkn_fn_string
# Info['VocabDict'] = VocabDict
Info['EmbedDict'] = EmbedDict
Info = pd.Series(Info)
Info

RecLevel                                                               P
RecLevelID                                                           PID
SynFld                                                         PatEcInfo
GrnName                                                          InfoGrn
Rec2FldList_Dict       {'P': ['age', 'basicInfo'], 'EC': ['DT_min', '...
prefix_ids                                                            []
focal_ids                                                          [PID]
rec_folder                                       data/ProcData/RecFolder
recfldgrn                                            P@PatEcInfo-InfoGrn
prefix_recfldgrn                                     P@PatEcInfo-InfoGrn
grain_tkn_fn_string    def grain_tkn_fn(df_input):\n    \n    '''\n  ...
EmbedDict              {'tkn': {'embed_type': 'CateEmbed', 'recfldgrn...
dtype: object

In [30]:
fldgrnInfo_folder = 'data/ProcData/FldGrnInfo'

if not os.path.exists(fldgrnInfo_folder): os.makedirs(fldgrnInfo_folder)

fullfldgrnInfo_file = os.path.join(fldgrnInfo_folder, prefix_recfldgrn + '.p')
print(fullfldgrnInfo_file)
Info.to_pickle(fullfldgrnInfo_file)

data/ProcData/FldGrnInfo/P@PatEcInfo-InfoGrn.p


In [31]:
from recfldgrn.datapoint import write_df_to_folders, RANGE_SIZE

fldgrnTensor_folder = f'data/ProcData/FldGrnTensor/{recfldgrn}'
if not os.path.exists(fldgrnTensor_folder): os.makedirs(fldgrnTensor_folder)

print(recfldgrn)
write_df_to_folders(recfldgrn, fldgrnTensor_folder, df_tensor_fnl, IDname = 'PID')

P@PatEcInfo-InfoGrn
